In [ ]:
import re, os, json

In [ ]:
os.makedirs("TTA_scaling_v2", exist_ok=True)

In [ ]:
folder_path = 'TTA_scaling'
folder_list = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [ ]:
KEEP_KEYWORDS = [
    # 일반 IT/서비스/플랫폼
    "서버", "클라이언트", "플랫폼", "서비스", "시스템", "모듈", "인터페이스", "API",
    "프로토콜", "패킷", "라우팅", "네트워크", "망", "단말", "게이트웨이", "포트",
    "클라우드", "가상화", "컨테이너", "쿠버네티스", "오케스트레이션",
    # 데이터/AI/STT
    "데이터", "DB", "데이터베이스", "스키마", "쿼리", "ETL",
    "인공지능", "AI", "모델", "학습", "추론", "파인튜닝", "STT", "음성 인식", "TTS",
    # 보안/암호/PKI
    "보안", "암호", "암호화", "복호화", "키 관리", "키 복구", "키 분배",
    "인증", "인가", "접근제어", "PKI", "전자서명", "토큰", "인증서",
    # 표준/문서/프로세스
    "표준", "규격", "규격서", "TTA", "RFC",
    "요구사항", "요구 사항", "요구 분석", "설계서", "명세서", "사양서",
    "제안요청서", "RFP", "견적", "가격 견적", "프로젝트", "과제", "사업자",
]

DROP_KEYWORDS = [
    # 농업/축산/동물/식물/수산 등
    "가금", "닭", "오리", "칠면조", "농가", "농업", "축산", "가축", "사육",
    "어류", "수산", "어민", "양식", "생태계", "야생동물", "동물", "식물",
    # 일반 교양/백과 느낌(필요시 추가)
    "세계사", "역사상", "민족", "종교", "신화", "철학", "문학", "예술사",
]

ENG_KEEP_PATTERN = re.compile(r"[A-Za-z]{2,}")  # PKI, API, TCP 등 영문 토큰 힌트

def should_keep(folder: str, question: str, answer: str) -> bool:
    """
    TTA 용어 1개(질문/답변)에 대해 KEEP/ DROP 여부를 판정하는 규칙 기반 함수
    """
    text = f"{question} {answer}"

    # 2) 명확 DROP 키워드가 들어가면 우선 DROP
    for kw in DROP_KEYWORDS:
        if kw in text:
            return False

    # 3) 도메인 KEEP 키워드가 하나라도 들어가면 KEEP
    for kw in KEEP_KEYWORDS:
        if kw in text:
            return True

    # 4) 영어 약어/용어가 포함되어 있으면 도메인 가능성이 높으니 KEEP로 본다
    if ENG_KEEP_PATTERN.search(text):
        return True

    # 5) 그 외는 기본적으로 DROP (필요하면 보수적으로 True로 바꿔도 됨)
    return False

In [ ]:
for folder in folder_list:
    target_folder = os.path.join("TTA_scaling_v2", folder)
    os.makedirs(target_folder, exist_ok=True)
    file_list = os.listdir(os.path.join(folder_path, folder))
    for file in file_list:
        print(os.path.join(folder_path, folder, file))
        with open(os.path.join(folder_path, folder, file), "r", encoding="utf-8") as f:    
            with open(os.path.join("TTA_scaling_v2", folder, "keep_" + file), 'w', encoding="utf-8") as f_keep:
                with open(os.path.join("TTA_scaling_v2", folder, "drop_"+ file), 'w', encoding="utf-8") as f_drop:
                    for line in f:
                        data = json.loads(line)
                        print(data["question"], data["answer"])
                        messages = {"messages": [{ "role": "user", "content": data["question"]}, { "role": "assistant", "content": data["answer"] }]}
                        if should_keep(folder, data["question"], data["answer"]):
                            f_keep.write(json.dumps(messages, ensure_ascii=False) + '\n')
                        else:
                            f_drop.write(json.dumps(messages, ensure_ascii=False) + '\n')